In [2]:
import scipy.stats
import numpy as np
import scipy.stats as ss
from matplotlib import pyplot as plt
import math 
import plotly.graph_objects as go
import pandas as pd 

In [65]:
class Petroleum_asset(object):
    def __init__(self, location, hc_type, phase, res_est, well_max_rate, well_cap_pot, n, init_year, period):
        
        self.location = location            #asset location (onshore or offshore)
        self.hc_type = hc_type              #asset hydrocarbon type (oil or gas)
        self.phase = phase                  #asset life cycle phase (exploration or development)
        self.res_est = res_est              #asset estimated recoverable reserves (mode value)
        self.well_max_rate = well_max_rate  #asset average maximum well rate (mode value)
        self.well_cap_pot = well_cap_pot    #asset well capacity potential 
        self.n = n                          #number of iterations
        self.init_year = init_year          #asset initial year to start the project 
        self.period = period                #project period (years)
        
        oil_price, gas_price = og_price(self.period, self.n)
        if self.hc_type == 'oil':
            self.price = oil_price
        else:
            self.price = gas_price
        
    def asset_evaluation(self):
        
        def pert(self, n, min, max, mode):

            # Calculating Beta-distribution parameters, based on the PERT parameters
            alpha = 2 * (max + 4 * mode - 5 * min) / (3 * (max - min)) * \
            (1 + 4 * (mode - min) * (max - mode) / (max - min)**2)

            beta = 2 * (5 * max - 4 * mode - min) / (3 * (max - min)) * \
            (1 + 4 * (mode - min) * (max - mode) / (max - min)**2)

            # Generating Beta-distribution (scipy.stats package)
            my_beta_distr = scipy.stats.beta.rvs(a = alpha, b = beta, size=n)

            # Converting Beta into PERT (scaling)
            pert = my_beta_distr * (max - min) + min

            return(pert)
        
 
        ##Function that estimates the reserves and total project capacity for each O&G asset 
        def fac_model (self):

            ###Reserves Estimates 
            res_m_on, res_m_off = 1.0, 2.0 #Mstb 
            res_min_fac_dev, res_max_fac_dev = 0.85, 1.5 #Mstb
            res_min_fac_exp, res_max_fac_exp = 0.2, 2 #Mstb
             

            #reserves in MM Boe
            if self.phase == 'exploration':
                if self.hc_type == 'oil':
                    if self.location == 'onshore':
                        res_mode = res_m_on*self.res_est
                    elif self.location == 'offshore':
                        res_mode = res_m_off*self.res_est

                if self.hc_type == 'gas':
                    if self.location == 'onshore':
                        res_mode = (res_m_on*self.res_est)
                    elif self.location == 'offshore':
                        res_mode = (res_m_off*self.res_est)

                res_min = res_min_fac_exp*res_mode
                res_max = res_max_fac_exp*res_mode
                reserves=pert(self, n=self.n, min=res_min, max=res_max, mode=res_mode)

            elif self.phase =='development':
                if self.hc_type == 'oil':
                    if self.location == 'onshore':
                        res_mode = res_m_on*self.res_est
                    elif self.location == 'offshore':
                        res_mode = res_m_off*self.res_est

                if self.hc_type == 'gas':
                    if self.location == 'onshore':
                        res_mode = (res_m_on*self.res_est)
                    elif self.location == 'offshore':
                        res_mode = (res_m_off*self.res_est)

                res_min = res_min_fac_dev*res_mode
                res_max = res_max_fac_dev*res_mode
                reserves=pert(self, n=self.n, min=res_min, max=res_max, mode=res_mode)

            ###Maximum capacity of the facility of the field 
            w_max_prod = (365/1000)* self.well_max_rate #well maximum production in Mb/y
            total_max_cap = w_max_prod * self.well_cap_pot / 2  #total asset maximum processing capacity in Mb/y, BSCF/y

            return (reserves, total_max_cap)
        
        #locd,bbd = fac_model (self)
        #print(locd)
        
        ##Function that estimates the years to start exploration - development - production
        def years_phase (self):
            if self.phase == 'exploration':
                expl_fac = np.array([1 for i in range(0, self.n)])  #if project is in exploration phase
                dev_fac = np.array([0 for i in range(0, self.n)])  #if project is in exploration phase
            elif self.phase == 'development':
                expl_fac = np.array([0 for i in range(0, self.n)]) #if project is in development phase
                dev_fac = np.array([1 for i in range(0, self.n)])  #if project is in development phase

            #length of exploration period 
            year_array = [2,3,4,5] 
            expl_prob1 = [0.2 for i in range (0,self.n)] 
            expl_prob2 = [0.2 for i in range (0,self.n)] 
            expl_prob3 = [0.3 for i in range (0,self.n)]
            expl_prob4 = [0.3 for i in range (0,self.n)]
            prob_array = [[expl_prob1[:][-1], expl_prob2[:][-1], expl_prob3[:][-1], expl_prob4[:][-1]]
                          for i in range(0, self.n)]
            expl_len = [int(np.random.choice(size = 1, a = year_array, p = prob_array[i])) 
                    for i in range(0, self.n)] * expl_fac 
            #print(expl_len)

            y_start_exp = self.init_year*expl_fac

            #years to start development 
            y_start_dev = self.init_year + expl_len 
            #print (y_start_dev)

            #length of development period 
            year_array1 = [1,2,3]
            dev_prob1 = [0.5 for i in range (0,self.n)] 
            dev_prob2 = [0.3 for i in range (0,self.n)] 
            dev_prob3 = [0.2 for i in range (0,self.n)] 
            prob_array1 = [[dev_prob1[:][-1], dev_prob2[:][-1], dev_prob3[:][-1]]for i in range(0, self.n)]
            dev_len = [int(np.random.choice(size = 1, a = year_array1, p = prob_array1[i])) 
                   for i in range(0, self.n)] 

            #years to start production 
            y_start_prod = y_start_dev + dev_len
            #print(y_start_prod)

            return (y_start_prod, y_start_exp, y_start_dev, expl_len)
        

        ##Production Forecast Model
        def production (self, fac_factors=1):
            ###Production forecast
            #n=10 #iterations
            #period = 31 #years 
            fac_factors = fac_model (self)
            #reserves = (2*10**6) * fac_factors[0] #Mstb
            #reserves = fac_factors[0] #MBoe
            reserves = (5*10**5)*fac_factors[0] #Mstb
            #reserves = (1*10**3)*fac_factors[0] #Mstb
            #prod_init = np.random.triangular(4.5,5,6,n) #initial production in Mbbls/d 
            #qi = reserves*(0.02)
            qi = reserves*(0.06)
            #qi = prod_init * (365/1000) #initial production in Mb/y 
            #t_to_plateau = np.random.uniform(0,54/12,self.n) #length of delay period, when production rate equals to 0
            t_to_plateau = np.random.uniform(24/12,36/12,self.n) 
            t_plateau = np.random.uniform(84/12,90/12,self.n) #length of development, when production rate rises from 0 to qi
            t_delay = np.random.uniform(0,2/12,self.n) #length of production plateau period, when production stays at qi
            a_factor = np.random.uniform(0.1,0.2,self.n) 
            production = np.array([[0 for i in range(0, self.n)] for k in range(0, self.period)])
            #diff = np.array([0 for i in range (0,n)])
            diff = np.zeros(self.n)
            years= years_phase (self)
            start_year = years[0]
            #N = int(np.mean(start_year))
            #production = 0
            for i in range (0,self.n):
                N=start_year[i]
                #j=1
                j=N
                total_prod = 0
                while (j<=t_delay[i]):
                    production [j,i] = 0
                    total_prod = total_prod + production [j,i] 
                    j=j+1
                while ((j-t_delay[i])<=t_to_plateau[i]):
                    rand = 0
                    production [j,i] = (qi[i]/t_to_plateau[i])*(j-t_delay[i])+rand 
                    total_prod = total_prod + production [j,i] 
                    j=j+1
                while ((j-t_delay[i]-t_to_plateau[i])<=t_plateau[i]):
                    rand = 0
                    production [j,i] = qi[i]+rand 
                    total_prod = total_prod + production [j,i] 
                    j=j+1
                #diff = reserves[i] - total_prod 
                diff[i]=reserves[i]-total_prod
                #total_max_cap = (fac_factors[1])*(10)
                total_max_cap = (fac_factors[1])*10
                while (diff[i]>total_max_cap):
                    rand = 0
                    dt = j-t_delay[i]-t_to_plateau[i]-t_plateau[i]
                    production [j,i]=qi[i]*math.exp(-a_factor[i]*dt) + rand 
                    total_prod = total_prod + production [j,i]
                   # diff = reserves[i] - total_prod
                    diff[i]=reserves[i]-total_prod

                    if j==30:
                        percent_prod = 100*total_prod/reserves[i] 
                        break 
                    j=j+1
                if i==self.n:
                    break 

            #prod_plot=np.transpose(production)
            prod_plot=np.transpose(production/(5*10**5))
            #years= years_phase (self)
            #start_year = years[0]
            #N = int(np.mean(start_year))
            #shape=np.shape(prod_plot)
            #padded_array = np.zeros((self.n,self.period+N))
            #padded_array[:shape[0], N:shape[1]+N] = prod_plot
            #modified_prod = np.array([padded_array[i][:-N] for i in range (0,self.n)])
            
            #prod_plot=np.transpose(production)
            #start_year= years_phase (self)
            #modified_prod = []
            #for i in range (0,self.n):
            #    N=start_year[i]-1
            #    prod_mod1 = np.pad(prod_plot[i], (N,0), 'constant')
            #    prod_mod = prod_mod1[:-N]
             #   modified_prod.append(prod_mod)
            return (prod_plot)
        
        prod = production(self)
        prod_mean = np.array([np.mean(prod.T[i]) \
                     for i in range(len(prod.T))])
        total_prod = np.sum(prod_mean)
        #print(total_prod)
        
        ##Reserves in MM Boe and total max capacity in Mb/year or BSCF/year
        reserves, capacity = fac_model (self)

        ##Number of injection and production wells (as a function of reserves)
        wells = np.array([
        [3, 5, 10, 15, 20, 25, 25, 40],
        [3, 4, 2, 4, 10, 15, 15, 30]
        ])
        res_vctr = [100, 200, 500, 1000, 2000, 5000, 10000, 20000]  #reserves limits in MM Boe
        n_inj_wells = [0 for i in range(0, self.n)]
        n_prod_wells = [0 for i in range(0, self.n)]
        for i in range (0,self.n):
            for k in range (0, 6):
                if reserves[i]>=res_vctr[k] and reserves[i]<=res_vctr[k+1]:
                    n_inj_wells[i] = wells[1][k]
                    n_prod_wells[i] = wells[0][k]

        ##Number of exploration and appraisal wells (constant for all projects)
        n_exp_wells = [3 for i in range(0, self.n)]
        n_app_wells = [2 for i in range(0, self.n)] #appraisal wells only if exploration was successful
        
        ##Capex Calculation
        years = years_phase (self)
        y_start_prod = years[0]
        y_start_exp = years[1]
        y_start_dev = years[2]
        expl_len = years [3]
        init_year = self.init_year

        q1 = np.percentile(reserves,20)
        q2 = np.percentile(reserves,50)
        

        unc_vctr = [0.5, 3]
        #fixed opex (onshore) = 0.005 - fixed opex(offshore) = 0.015
        #variable opex(onshore) = - variable opex (offshore) = 
        opex_sc = 6
        opex_vctr = pert(self, n=self.n, min=opex_sc*0.7, max=opex_sc*1.6, mode=opex_sc)


        ###CO2 Emission Model
        modified_prod = production(self)
        co2_base = 30 #kgCO2/toe
        emission_int = np.array([[0 for i in range(0, self.period)] for k in range(0, 1)])
        #emission_int = np.array([[0 for i in range(0, period)]])
        for i in range (0,self.n):
            peak_prod = max(modified_prod[i])
            share_prod = np.array([(modified_prod[i][k]/peak_prod) for k in range (0,self.period)]) 
            emission_share = np.array([(1.0096*(share_prod[k])**-0.65) for k in range(0,self.period)])
            emission_share = np.where(emission_share == float('inf'), 0, emission_share)
            emission = np.array([emission_share[k]*co2_base for k in range (0,self.period)])
            emission_int = np.append(emission_int,[emission], axis=0)
        #print(emission_int)
        co2_emission = np.delete(emission_int, 0, 0) #kgCO2/toe

        co2 = np.array([[0 for i in range(0, self.n)] for k in range(0, 1)])
        for k in range (0,self.period-1):
            total = np.array([modified_prod[i][k]*co2_emission[i][k]*136 for i in range (0,self.n)])
            co2 = np.append(co2, [total], axis=0) #tCO2 per year
        #print(new)
        co2_price = 20 #$tC02
        co2_tax_mean = np.array([np.mean(co2[i])*co2_price*10**-6 \
                             for i in range(len(co2))])
        #print(co2_tax_mean)



        if self.phase == 'exploration':
            expl_fac = np.array([1 for i in range(0, self.n)]) 
        else:
            expl_fac = np.array([0 for i in range(0, self.n)]) 
        dev_fac = np.array([1 for i in range(0, self.n)])

        
        capex_seis = np.random.triangular(8, 10, 15, self.n)
        if self.location == 'offshore':
            cap_fact = 2
        else:
            cap_fact = 1
        cap1 = np.random.triangular(130, 160, 190, self.n)
        cap2 = np.random.triangular(120, 150, 190, self.n)
        capex_exp = np.array([cap_fact*cap1[i]*n_exp_wells[i] for i in range (0,self.n)])
        capex_inj = np.array([cap1[i]*n_inj_wells[i] for i in range (0,self.n)])
        capex_app = np.array([cap2[i]*n_app_wells[i] for i in range (0,self.n)])
        capex_prod = np.array([cap_fact*cap2[i]*n_prod_wells[i] for i in range (0,self.n)])
        #fixed_opex = np.random.triangular(1.3, 1.5, 1.8, self.n)
        #fixed_opex = np.random.triangular(13, 15, 18, self.n)
        fix_opex = np.random.triangular(10, 12, 15, self.n)
        if self.hc_type == 'gas':
            uplift_fac = 1.5
            GOR = 9
        else:
            uplift_fac = 1
            GOR = 1
        fixed_opex = np.array([uplift_fac*fix_opex[i]*(n_prod_wells[i]+n_app_wells[i]) for i in range (0,self.n)])
        var_opex = np.random.triangular(10, 15, 20, self.n)

        #import length of exploration phase (expl_len) from years function
        capex_sc = 1
        capex2_time = np.array([0 for i in range(0, self.n)])
        capex2 = np.array([0 for i in range(0, self.n)])
        capex3 = np.array([0 for i in range(0, self.n)])
        hc_price = np.array([50 for i in range(0, self.period)])

        for i in range (0,self.n):
            #capex1[i] = capex_seis[i]*expl_fac[i]
            capex2_time [i] = self.init_year + (expl_len[i]/2)
            if (reserves [i])<q1:
                capex2[i] = 0
                #seis_succ = 0
                #dril_succ = 0
            else:
                capex2[i] = capex_exp[i]*expl_fac[i]
                if (reserves[i])<q2: 
                    capex3[i] = 0
                else:
                    capex3[i] = capex_prod[i] + capex_inj[i]+capex_app[i]

        time = [0 + i for i in range(0, self.period)]
        capex = np.array([[0 for i in range(0, self.n)] for k in range(0, 1)])
        btcf = np.array([[0 for i in range(0, self.n)] for k in range(0, 1)])
        discount_factors=np.zeros(self.period)
        revenues = np.array([[0 for i in range(0, self.n)] for k in range(0, 1)])
        opex = np.array([[0 for i in range(0, self.n)] for k in range(0, 1)])
        total = np.array([1 / ((1 + 0.45)**(time[i] - 0.5)) for i in range(0, self.period)])
        cc1=np.array([[0 for i in range(0, self.n)] for k in range(0, 1)])
        for k in range (0, self.period-1): 
            expl_cap1 = np.array(
                [
                    0 
                    if (time[k] < y_start_exp[i]) or (time[k]>y_start_dev[i])
                    else (capex_seis[i])
                    for i in range (0,self.n)
                    ] * expl_fac
                )

            expl_cap2 = np.array(
                [
                    0 
                    if (time[k] < y_start_exp[i]) or (time[k]>capex2_time[i])
                    else (capex2[i])
                    for i in range (0,self.n)
                    ] * expl_fac
                )


            dril_cap = np.array(
                [
                    0 
                    if (time[k] < y_start_dev[i]) or (time[k]>y_start_prod[i])
                    else (capex3[i] / (y_start_prod[i] - y_start_dev[i]))
                    for i in range (0,self.n)
                    ] * dev_fac
                )
            cc1 = np.append(cc1, [dril_cap], axis=0)

            total = expl_cap1 + expl_cap2 + dril_cap
            #total=expl_cap1 + expl_cap2 
            capex = np.append(capex, [total], axis = 0)


            #total2 = np.array([modified_prod[i][k]* (hc_price[k]) for i in range (0,self.n)])
            total2 = np.array([GOR*modified_prod[i][k]* (self.price[k][i]) for i in range (0,self.n)])
            revenues = np.append(revenues, [total2], axis=0)
            #total3 = np.array([modified_prod[i][k]* (var_opex[i]) for i in range (0,self.n)])
            total3 = np.array([modified_prod[i][k]* (var_opex[i])+fixed_opex[i] for i in range (0,self.n)])
            opex = np.append(opex, [total3], axis=0)

            cash_flow = - capex[k+1] + revenues[k+1] - opex[k+1]
            btcf = np.append(btcf, [cash_flow], axis=0)

        tax_rate = 0.56 
        int_rate = 0.08
        atcf = btcf*(1-tax_rate)
        discount_factors=np.zeros(self.period)
        PW = np.array([[0 for i in range(0, self.n)] for k in range(0, 1)])
        for k in range (1,self.period):
            discount_factors[k]= 1/((1+int_rate)**(k))
            total = atcf[k,:]*discount_factors[k] 
            PW = np.append(PW, [total], axis=0)

        npv = np.array([np.sum(PW[:,i]) for i in range (0,self.n)])
        npv_mean = np.mean(npv)

        #print(npv_mean)
    

    ###Storing the simulation results 
        annual_prod = modified_prod.T
        trye = co2_emission.T
        project_data = [self.hc_type, self.phase, self.location,
                        reserves, annual_prod, co2, capex, opex, revenues, PW, npv, btcf]
        self.result = project_data 
        self.reserves = Petroleum_asset.data_tables(self, data_type = project_data, col = 3)
        self.annual_prod = Petroleum_asset.data_tables(self, data_type = project_data, col = 4)
        self.co2_emission = Petroleum_asset.data_tables(self, data_type = project_data, col = 5)
        self.capex = Petroleum_asset.data_tables(self, data_type = project_data, col = 6)
        self.opex = Petroleum_asset.data_tables(self, data_type = project_data, col = 7)
        self.revenues = Petroleum_asset.data_tables(self, data_type = project_data, col = 8)
        self.PW = Petroleum_asset.data_tables(self, data_type = project_data, col = 9)
        self.npv = Petroleum_asset.data_tables(self, data_type = project_data, col = 10)
        self.btcf = Petroleum_asset.data_tables(self, data_type = project_data, col = 11)
        
    def data_tables(self, data_type, col):
        table = pd.DataFrame(data_type[col])
        return table
    
    def asset_results(self, data=False):
        if data:
            #return(self.PW, self.revenues, self.opex, self.capex, self.co2_emission, self.annual_prod, self.npv)
            return(self.result)
        else:
            val = int(input("Simulation Results:\n"
                            "\n"
                            "1    All Results \n"
                            "2    Present Worth \n"
                            "3    Revenues \n"
                            "4    Opex \n"
                            "5    Capex \n"
                            "6    CO2 Emission \n"
                            "7    Annual Production \n"
                            "8    NPV \n"
                            "9    Reserves \n"
                            "10   Undiscounted Net Cash Flow \n"
                            "\n"
                            "Enter Data Type Needed: "))
            
            if val == 1:
                return self.result
            elif val == 2:
                return self.PW
            elif val == 3:
                return self.revenues
            elif val == 4:
                return self.opex
            elif val == 5:
                return self.capex
            elif val == 6:
                return self.co2_emission
            elif val == 7:
                return self.annual_prod
            elif val == 8:
                return self.npv
            elif val == 9:
                return self.reserves
            elif val == 10:
                return self.btcf
         
    def asset_plot(self):
        val = int(input("Plot Type:\n"
                        "\n"
                        "1    Production Profile \n"
                        "2    Cash Flow (annual mean) \n"
                        "3    Undiscounted Net Cash Flow (P90-P10-Mean) \n"
                        "4    CO2 Emission (annual mean) \n"
                        "5    Present Worth (P90-P10-Mean) \n"
                        "6    NPV Distribution \n"
                        "7    Reserves Distribution \n"
                        "8    Cash Flow Distribution of Year X  \n"
                        "\n"
                        "Enter Plot Needed: "))
        
        if val == 1:
            fig_f2 = go.Figure()
            for i in range (0,self.n):
                fig_f2.add_scatter(y=self.annual_prod[i])
                fig_f2.update_layout(title="Production Profile Distribution",
                                  yaxis_title="Annual Oil/Gas Production (Mstb or Bscf)",
                                  xaxis_title="Years",
                                  legend_title="Iteration",
                                  font=dict(family="Courier New, monospace",size=12,color="RebeccaPurple")
                                 )
            fig_f2.show()
            
            fig_f1 = go.Figure()
            fig_f1.add_scatter(y=self.annual_prod.T.quantile(0.1), name = 'P10')
            fig_f1.add_scatter(y=self.annual_prod.mean(axis=1), name = 'Mean')
            fig_f1.add_scatter(y=self.annual_prod.T.quantile(0.9), name = 'P90')
            fig_f1.update_layout(title="Production Profile",
                              yaxis_title="Annual Oil/Gas Production (Mstb or Bscf)",
                              xaxis_title="Years",
                              font=dict(family="Courier New, monospace",size=12,color="RebeccaPurple")
                             )
            fig_f1.show()
        
        if val == 2:
            capex_mean = self.capex.mean(axis=1)
            revenues_mean = self.revenues.mean(axis=1)
            opex_mean = self.opex.mean(axis=1)
            x1=-opex_mean
            x2=-capex_mean
            x3= revenues_mean
            # lists to dict
            my_dict = {'opex': x1, 'capex': x2, 'revenues':x3}
            # dict to df
            result_df = pd.DataFrame(my_dict)
            result_df

            fig, ax = plt.subplots(1, figsize=(16, 8))
            plt.bar(result_df.index, result_df['revenues'], color = '#337AE3', width =0.5)
            #plt.bar(result_df.index, result_df['co2 tax'], color = '#5E96E9', width =0.5)
            plt.bar(result_df.index, result_df['capex'], bottom = result_df['opex'], color = '#DB4444', width =0.5)
            plt.bar(result_df.index, result_df['opex'], color = '#E17979', width =0.5)
            # remove spines
            ax.spines['right'].set_visible(False)
            ax.spines['left'].set_visible(False)
            ax.spines['top'].set_visible(False)
            ax.spines['bottom'].set_visible(False)
            #grid
            ax.set_axisbelow(True)
            ax.yaxis.grid(color='gray', linestyle='dashed', alpha=0.7)
            # x ticks
            years = [0 + i for i in range(0, self.period)]
            plt.xticks(result_df.index , labels = years)
            # title and legend
            legend_label = ['revenues', 'capex', 'opex']
            plt.legend(legend_label, ncol = 4, bbox_to_anchor=([1, 1.05, 0, 0]), frameon = False)
            plt.title('Cash Flow Analysis', loc='left')
            plt.xlabel('years')
            plt.ylabel('USD (millions)')
            plt.show()
            
        if val == 3: 
            fig_f3 = go.Figure()
            fig_f3.add_scatter(y=self.btcf.T.quantile(0.1), name = 'P10')
            fig_f3.add_scatter(y=self.btcf.mean(axis=1), name = 'Mean')
            fig_f3.add_scatter(y=self.btcf.T.quantile(0.9), name = 'P90')
            fig_f3.update_layout(title="Undiscounted Net Cash Flow",
                              yaxis_title="USD (million)",
                              xaxis_title="Years",
                              font=dict(family="Courier New, monospace",size=12,color="RebeccaPurple")
                             )
            fig_f3.show()
            
        if val == 4:
            fig_f4 = go.Figure()
            fig_f4.add_scatter(y=self.co2_emission.T.quantile(0.1), name = 'P10')
            fig_f4.add_scatter(y=self.co2_emission.mean(axis=1), name = 'Mean')
            fig_f4.add_scatter(y=self.co2_emission.T.quantile(0.9), name = 'P90')
            fig_f4.update_layout(title="CO2 Emission",
                              yaxis_title="t CO2",
                              xaxis_title="Years",
                              font=dict(family="Courier New, monospace",size=12,color="RebeccaPurple")
                             )
            fig_f4.show()
            
            
            ### production and emission intensity graph
            ## should put trye instead of co2 above 
            annual_prod_mean = self.annual_prod.mean(axis=1)
            x1=annual_prod_mean*0.136
            my_dict = {'production': x1}
            # dict to df
            result_df = pd.DataFrame(my_dict)
            result_df
            
        if val == 4.1: 

            ### (not part of the code - just maybe show the graph in thesis)
            #fig, ax = plt.subplots(1, figsize=(16, 8))
            plt.bar(result_df.index, result_df['production'], color = '#337AE3', width =0.5)
            ax.spines['right'].set_visible(False)
            ax.spines['left'].set_visible(False)
            ax.spines['top'].set_visible(False)
            ax.spines['bottom'].set_visible(False)
            #grid
            ax.set_axisbelow(True)
            ax.yaxis.grid(color='gray', linestyle='dashed', alpha=0.7)
            # x ticks
            years = [0 + i for i in range(0, self.period)]
            plt.xticks(result_df.index , labels = years)
            # title and legend
            legend_label = ['production']
            plt.legend(legend_label, ncol = 4, bbox_to_anchor=([1, 1.05, 0, 0]), frameon = False)
            plt.title('Production and emission per unit', loc='left')
            plt.xlabel('years')
            plt.ylabel('Million toe')
            #plt.show()
            
            re = self.co2_emission.mean(axis=1).plot(secondary_y=True, label = 'Emission per unit', color = 'green')
            re.legend(fontsize=12, frameon=False, bbox_to_anchor=([1, 1.05, 0, 0.05]))
            re.legend('emission per unit', fontsize=12)
            #re.set_ylim(30,70)
            re.set_ylabel('kg CO2/toe', fontsize=16)
            fig = re.get_figure()
        
            #plt.show()
            
        if val == 5:
            fig_f3 = go.Figure()
            fig_f3.add_scatter(y=self.PW.T.quantile(0.1), name = 'P10')
            fig_f3.add_scatter(y=self.PW.mean(axis=1), name = 'Mean')
            fig_f3.add_scatter(y=self.PW.T.quantile(0.9), name = 'P90')
            fig_f3.update_layout(title="Net Cash Flow",
                              yaxis_title="USD (million)",
                              xaxis_title="Years",
                              font=dict(family="Courier New, monospace",size=12,color="RebeccaPurple")
                             )
            fig_f3.show()
            
        
        if val == 6:
            import plotly.express as px
            hist_reserves = px.histogram(self.npv, histnorm='probability', nbins=20)
            #hist_reserves = px.histogram(self.npv, nbins=10)
            hist_reserves.update_layout(xaxis_title="NPV (USD million)", yaxis_title="Normalized Frequency")
            hist_reserves.show()
            
        if val == 7:
            import plotly.express as px
            hist_reserves = px.histogram(self.reserves, histnorm='probability')
            hist_reserves.update_layout(xaxis_title="Reserves (Mstb)", yaxis_title="Normalized Frequency")
            hist_reserves.show()



#define a function that calculates the oil and gas prices 
def og_price (period, n):
    #Mean reverting parameters derived from historical HC prices 
    dt = 1 #time step for oil and gas in years  
    oil_floor = 8 #oil price floor in USD/bbl
    gas_floor = 0.8 #gas price floor in USD/Mscf
    oil_sd = 3   #oil volatility in USD/bbl
    gas_sd = 0.7 #gas volatility in USD/Mscf
    oil_half = 4 #oil half-life in years 
    gas_half = 8 #gas half-life in years 
    oil_ini_price = 40 #oil initial price in USD/bbl
    gas_ini_price = 2.3 #gas initial price in USD/Mscf
    oil_mean_price = 70 #oil long term mean price in USD/bbl
    gas_mean_price = 5 #gas long term mean price in USD/Mscf 
    
    
    def hc_price (period, dt, floor, sd, life_half, ini_price, mean_price):
        speed = math.log(2) / (life_half * dt) #mean reversion rate
        price_og = np.zeros(period+1) #zeros array to store the price generated 
        price_og[0] = ini_price #set the first input to the array to be equal to the initial oil or gas price
        #epsilon = np.random.normal (0,1,101) #random variable with standard normal distribution 
        
        #Correlation between oil and gas prices using Cholesky decomposition
        x_uncor = np.random.normal(0, 1, (2, 101))
        cov = np.array([[ 1.0, 0.63], 
                        [ 0.63, 1.0]])  #oil and gas correlation matrix 
        L = np.linalg.cholesky(cov)
        x_cor = np.dot(L, x_uncor)
        corr_simulated = pd.DataFrame(x_cor).T.corr()
        std_ = np.sqrt(np.diag(cov))
        corr_empirical = cov / np.outer(std_, std_)
        
        for i in range(1, period+1):
            price_og[i] = max(floor, price_og[i-1] +(mean_price - price_og[i-1]) * dt * speed + 
                              sd * math.sqrt(dt) * x_cor[1][i])
        return (price_og)
    
    
    #Calculate the oil price using the hydrocarbon price function (hc_price)
    oil_price = np.array([hc_price(period, dt, oil_floor, oil_sd, oil_half, oil_ini_price, oil_mean_price) 
                          for i in range(0, n)])
    oil_price = np.transpose(oil_price)
    
    #Calculate the gas price using the hydrocarbon price function (hc_price)
    gas_price = np.array([hc_price(period, dt, gas_floor, gas_sd, gas_half, gas_ini_price, gas_mean_price) 
                          for i in range(0, n)])
    gas_price = np.transpose(gas_price)

    return (oil_price, gas_price)

        
        
        
        
        
        
    

In [66]:
ij=Petroleum_asset(location='onshore', hc_type='oil', phase='exploration', 
                res_est=100, well_max_rate=40, well_cap_pot=0.5, n=10, init_year=0, period=31)
ij.asset_evaluation()
#ij.asset_results(data=False)
#ij.asset_plot()

<ipython-input-65-82108be07725>:194: RuntimeWarning:

overflow encountered in long_scalars

<ipython-input-65-82108be07725>:274: RuntimeWarning:

divide by zero encountered in double_scalars



In [67]:
class Wind_farm_asset(object):
    def __init__(self, location, n_turbine, const_sc, perc_subs, period, n):
        
        self.location = location            #asset location (onshore or offshore)
        self.n_turbine = n_turbine          #asset number of turbines (30 or 60)
        self.const_sc = const_sc            #asset construction sceheme (10/80/10 or 30/60/10)
        self.perc_subs = perc_subs          #asset government percentage of subsidies 
        self.n = n                          #number of iterations
        self.period = period                #project period (years)
        
        
    def asset_evaluation(self):
        
        def pert(self, n, min, max, mode):

            # Calculating Beta-distribution parameters, based on the PERT parameters
            alpha = 2 * (max + 4 * mode - 5 * min) / (3 * (max - min)) * \
            (1 + 4 * (mode - min) * (max - mode) / (max - min)**2)

            beta = 2 * (5 * max - 4 * mode - min) / (3 * (max - min)) * \
            (1 + 4 * (mode - min) * (max - mode) / (max - min)**2)

            # Generating Beta-distribution (scipy.stats package)
            my_beta_distr = scipy.stats.beta.rvs(a = alpha, b = beta, size=n)

            # Converting Beta into PERT (scaling)
            pert = my_beta_distr * (max - min) + min

            return(pert)
        
        wacc=0.08

        #Installed capacity (MW) - 2.3 MW turbines 
        inst_cap = self.n_turbine*2.3 


        #Yearly production degradation (%)
        #energy_prod_deg = 0.005

        #Volume of capital cost (Y1-Y2-Y3)
        if self.const_sc == '10/80/10':
            cap_cons_1 = 0.1 
            cap_cons_2 = 0.8
            cap_cons_3 = 0.1 

        elif self.const_sc == '30/60/10':
            cap_cons_1 = 0.3 
            cap_cons_2 = 0.6
            cap_cons_3 = 0.1 
            

        #Capex and Opex 
        if self.location == 'onshore':
            opex = pert (self, n=self.n ,min=0.01, max=0.035, mode=0.015) #($/kWh)
            capex_var = 1000000*inst_cap * pert (self, n=self.n, min=1.2, max=2.29, mode=1.8)      #(m$/MW)
            capex_fix = pert(self, self.n, min=36, max=75, mode=50)
            capex = 0.73*capex_var + 0.27*capex_fix

        elif self.location == 'offshore':
            opex = pert (self, self.n, min=0.015, max=0.048, mode=0.03) #($/kWh)
            capex_var = 1000000*inst_cap*pert (self, n.self, min=2.29, max=5.42, mode=3.5)    #(m$/MW) 
            capex_fix = pert(self, self.n, min=43.2, max=90, mode=72)
            capex = 0.73*capex_var + 0.27*capex_fix

        #Annual subsidies and Capex 
        total_subs = capex*self.perc_subs
        annual_cap = np.zeros((self.n,self.period))
        annual_subs = np.zeros((self.n,self.period))
        repower = np.array([0.88*10**6]*self.n)
        for j in range (0,self.n):
            for i in range (0,3):
                annual_subs[j,i] = total_subs[j]/3
            annual_cap[j,0] = capex[j]*cap_cons_1
            annual_cap[j,1] = capex[j]*cap_cons_2
            annual_cap[j,2] = capex[j]*cap_cons_3
            annual_cap[j,21] = inst_cap*repower[j]

        #Annual energy production 
        purch_price = pert(self, self.n, min=0.08, max=0.14, mode=0.1068) #Purchase price of energy ($/kWh)
        energy_cap = pert(self, (self.n, self.period), min=0.5, max=0.85, mode=0.65) #Energy production capacity (%)
        energy_prod_theo = (self.n_turbine)*pert(self, (self.n, self.period), 
                                                 min=6500, max=10000, mode=8000) #Annual theoretical production (MWh)
        annual_ener_prod = energy_prod_theo*energy_cap #(in MWh)
        for i in range (0,3):
            annual_ener_prod[:,i] = 0

        #Annual revenues and Opex from energy production 
        purch_price_inc = 0.03  #purchase price yearly increment
        opex_inc = 0.015  #Annual increment in Opex
        annual_purch_price = np.zeros((self.n, self.period))
        annual_opex = np.zeros((self.n, self.period))
        for j in range (0,self.n):
            annual_purch_price[j,3]=purch_price[j]
            annual_opex[j,3]=opex[j]
            for i in range (4,self.period):
                annual_purch_price[j,i] = annual_purch_price[j,i-1]*(1+purch_price_inc)
                annual_opex[j,i] = annual_opex[j,i-1]*(1+opex_inc)
        annual_rev = annual_purch_price*annual_ener_prod*1000
        annual_cost_opex = annual_opex*annual_ener_prod*1000


        #Annual depreciation 
        annual_dep = np.zeros((self.n, self.period))
        for j in range (0,self.n): 
            for i in range (4,self.period):
                annual_dep [j,i] = capex[j]/(self.period-3)

        #BTCF and ATCF
        sv=pert(self, self.n, min=500000, max=1000000, mode=750000)
        BTCF=np.zeros((self.n, self.period))
        BTCF = annual_subs + annual_rev - annual_cost_opex - annual_cap
        BTCF[:,-1]+=sv

        TI=BTCF-annual_dep                            
        TI[:,0]=[0]     
        tax_rate = 0.4
        T=-tax_rate*TI      

        ATCF=BTCF+T  
        discount_factors=np.zeros(self.period)
        for k in range (0,self.period):
            discount_factors[k]= 1/((1+wacc)**(k))
        PW = sum((ATCF * discount_factors).T)
        PW = PW*(10**-6)

        ATCF_array = []
        for k in range (0,self.period):
            mean_ATCF = np.mean(ATCF[:,k])
            ATCF_array.append(mean_ATCF) #Annual Cash Flow 

        #CO2 Emission from Wind Farms
        co2_coef = pert (self, self.n, min=0.005, max=0.0082, mode=0.006) #(kg/kWh)
        co2_emission = np.zeros((self.n, self.period))
        for k in range (0,self.n):
            co2_emission[k,:]=co2_coef[k]
        annual_co2_emission = co2_emission*annual_ener_prod*10**-3
        
        ###Storing the simulation results 
        project_data = [self.n_turbine, self.location,
                        annual_co2_emission.T, PW, BTCF.T, (annual_cap*10**-6).T, (annual_ener_prod*10**-6).T]
        self.result = project_data 
        self.co2_emission = Wind_farm_asset.data_tables(self, data_type = project_data, col = 2)
        self.npv = Wind_farm_asset.data_tables(self, data_type = project_data, col = 3)
        self.btcf = Wind_farm_asset.data_tables(self, data_type = project_data, col = 4)
        self.capex = Wind_farm_asset.data_tables(self, data_type = project_data, col = 5)
        self.energy_prod = Wind_farm_asset.data_tables(self, data_type = project_data, col = 6)
        
    def data_tables(self, data_type, col):
        table = pd.DataFrame(data_type[col])
        return table
    
    def asset_results(self, data=False):
        if data:
            return(self.result)
        else:
            val = int(input("Simulation Results:\n"
                            "\n"
                            "1    All Results \n"
                            "2    NPV \n"
                            "3    Capex \n"
                            "4    CO2 Emission \n"
                            "5    Annual Energy Production \n"
                            "\n"
                            "Enter Data Type Needed: "))
            
            if val == 1:
                return self.result
            elif val == 2:
                return self.npv
            elif val == 3:
                return self.capex
            elif val == 4:
                return self.co2_emission
            elif val == 5:
                return self.energy_prod

In [68]:
ik=Wind_farm_asset(location='onshore', n_turbine=60, const_sc='10/80/10', 
                   perc_subs=0.45, period=30, n=10)
ik.asset_evaluation()
#ik.asset_results(data=False)

In [ ]:
class 